In [147]:
import time
import astroquery.heasarc
from astropy.coordinates import SkyCoord
import astropy.units as u

from astroquery.jplhorizons import Horizons
import numpy as np

from collections import defaultdict

from astropy.time import Time

import getpass

import logging

from oda_api.api import DispatcherAPI

import matplotlib.pyplot as plt
from astropy.io import fits

In the first cell, we define our query function. This is to be copied and pasted each time we want to query science windows. 

In [164]:
Heasarc = astroquery.heasarc.Heasarc()

def get_scw_list(ra_obj, dec_obj,radius,start_date,end_date ):
    R = Heasarc.query_region(
            position = SkyCoord(ra_obj, dec_obj, unit='deg'),
            radius = f"{radius} deg",
            catalog = 'intscw', # 'mission' has been deprecated, use catalog instead
            time = start_date + " .. " + end_date,
            good_isgri = ">1000",
        )
    R.sort('scw_id') # changed SCW_ID to scw_id
    R['scw_id', 'scw_ver', 'start_date', 'end_date', 'ra', 'dec'].pprint()
    return R['scw_id'], R['scw_ver'], R['start_date'], R['end_date']  # changed SCW_VER to scw_ver


assert astroquery.__version__ >= '0.4.2.dev6611'
assert 'isdc' in astroquery.heasarc.Conf.server.cfgtype

columns = Heasarc.list_columns(catalog_name='intscw')
columns.sort('name')
columns[:].pprint(align='<')

      name      ...  unit 
--------------- ... ------
data_in_heasarc ...       
data_size       ... byte  
dec             ... degree
end_date        ... mjd   
good_isgri      ... s     
good_jemx1      ... s     
good_jemx2      ... s     
good_omc        ... s     
good_picsit     ... s     
good_spi        ... s     
obs_id          ...       
obs_type        ...       
pi_name         ...       
ra              ... degree
scw_id          ...       
scw_type        ...       
scw_ver         ...       
start_date      ... mjd   
status          ...       


In this second cell, define our source or region to query.
What we want to do for Jupiter is find Jupiter's time-dependent position and query that for a given search radius and time interval. We use astroquery's JPL Horizons module.

In [165]:
epochs = {'start':'2004-01-01', 'stop':'2004-02-01', 'step':'1d'}

# Jupiter's ID in Horizons is 599
# We use @earth as we want geocentric coordinates
jupiter = Horizons(id='599', location='@0',epochs=epochs) # 599 is Jupiter's ID in this module
eph = jupiter.ephemerides()
#print(eph)

ra_values = eph['RA']  # in degrees
dec_values = eph['DEC']  # in degrees
print(eph[['datetime_str', 'RA', 'DEC']])

# let's calculate the mean change in Jupiter's position in the sky per day
coords = SkyCoord(ra=np.array(ra_values) * u.deg, dec=np.array(dec_values) * u.deg, frame='icrs')
angular_separations = coords[:-1].separation(coords[1:])
mean_daily_change = np.mean(angular_separations.to(u.arcmin))
print(f"Mean change in Jupiter's position per day: {mean_daily_change:.3f}")


   datetime_str       RA      DEC  
       ---           deg      deg  
----------------- --------- -------
2004-Jan-01 00:00 161.08537 9.20066
2004-Jan-02 00:00 161.15788 9.17256
2004-Jan-03 00:00 161.23037 9.14445
2004-Jan-04 00:00 161.30285 9.11632
2004-Jan-05 00:00  161.3753 9.08818
2004-Jan-06 00:00 161.44773 9.06003
2004-Jan-07 00:00 161.52015 9.03188
2004-Jan-08 00:00 161.59254  9.0037
2004-Jan-09 00:00 161.66493 8.97552
2004-Jan-10 00:00  161.7373 8.94732
              ...       ...     ...
2004-Jan-23 00:00 162.67654 8.57972
2004-Jan-24 00:00 162.74868 8.55136
2004-Jan-25 00:00  162.8208 8.52299
2004-Jan-26 00:00  162.8929 8.49461
2004-Jan-27 00:00 162.96499 8.46622
2004-Jan-28 00:00 163.03705 8.43782
2004-Jan-29 00:00 163.10909 8.40941
2004-Jan-30 00:00 163.18113 8.38099
2004-Jan-31 00:00 163.25315 8.35255
2004-Feb-01 00:00 163.32516  8.3241
Length = 32 rows
Mean change in Jupiter's position per day: 4.608 arcmin


Error due to seperation between INTEGRAL and Earth.

In [166]:
D = 150e3 # max distance between INTEGRAL and Earth (~ 150 000 km)
r = 4 * 149597870.7 # min distance between Jupiter and Earth is ~ 4 AU 

error = D/r * 206265/60 # convert to arcmin
print("Angular error is ", error, " arcmin") # below 1', insignificant (FWHM is 12' for INTEGRAL)

Angular error is  0.8617477267341882  arcmin


Retrieve the science windows. Make sure to choose ones that are of pointing type (id ends with 0).

In [167]:
scw_ids = []
scw_versions = []
scw_start_times = []
scw_end_times = []

# Loop through the ephemeris
for row in eph:
    ra = float(row['RA'])
    dec = float(row['DEC'])
    radius = 8.0 # search radius in degrees
    scw_id, scw_ver, scw_start, scw_end = get_scw_list(ra, dec, radius, '2004-01-01T00:00:00', '2004-02-01T00:00:00')
    scw_ids.extend(scw_id)
    scw_versions.extend(scw_ver)
    scw_start_times.extend(scw_start)
    scw_end_times.extend(scw_end)

# filter by type
scw_ids = [scw_id for scw_id in scw_ids if int(scw_id) % 10 == 0]
scw_versions = [scw_versions[i] for i in range(len(scw_ids)) if int(scw_ids[i]) % 10 == 0]
scw_start_times = [scw_start_times[i] for i in range(len(scw_ids)) if int(scw_ids[i]) % 10 == 0]
scw_end_times = [scw_end_times[i] for i in range(len(scw_ids)) if int(scw_ids[i]) % 10 == 0]

print("Found ", len(scw_ids), " pointing science windows")

   scw_id    scw_ver    start_date        end_date         ra      dec   
                            d                d            deg      deg   
------------ ------- ---------------- ---------------- --------- --------
007600040171     001 52788.5195513002 52788.5478730735 166.73833 12.52730
008200200021     001 52806.2710191345 52806.2830446035 167.28118 10.47886
026601100010     001 53358.8256557167 53358.8325191468 156.62770 10.60867
026601100021     001 53358.8325191468 53358.8338501662 156.45857 10.20109
026601110010     001 53358.8338501662 53358.8443131356 156.11153  9.37400
026601110022     001 53358.8443131356 53358.8546372161 156.18837  9.36603
026601110031     001 53358.8546372161 53358.8737460243 167.04653  5.20160
026700000210     001 53359.4028435524 53359.4254130089 154.57039  5.04156
026700000221     001 53359.4254130089  53359.426443102 154.56520  5.03179
026700000230     001  53359.426443102 53359.4493713549 154.56081  5.02164
         ...     ...              ... 

We then pick any science window(s) we want. Let us first group the ScWs by revolutions and pointings. This allows us to, for example, average/stack the images over pointings or over ScWs for a given pointing. This could be useful for mid-term timescale variability. 

In [168]:
grouped_scws = defaultdict(lambda: defaultdict(list))

scw_ids = list(set(scw_ids))

# Group by Revolution (RRRR) and then by Pointing (PPPP)
for scw in scw_ids:
    scw_str = str(scw)
    revolution = scw_str[:4]  # Extract RRRR
    pointing = scw_str[4:8]   # Extract PPPP
    grouped_scws[revolution][pointing].append(scw)

grouped_scws = sorted(grouped_scws.items(), key=lambda x: (x[0], sorted(x[1].keys()))) # sort

print(grouped_scws)
for rev, pointings in list(grouped_scws):
    print(f"Revolution: {rev}")
    for pointing, scws in pointings.items():
        print(f"  Pointing: {pointing}, # of ScWs: {len(scws)}")

[('0266', defaultdict(<class 'list'>, {'0110': ['026601100010'], '0111': ['026601110010']})), ('0267', defaultdict(<class 'list'>, {'0000': ['026700000210', '026700000230']})), ('0432', defaultdict(<class 'list'>, {'0065': ['043200650010'], '0064': ['043200640010']})), ('0917', defaultdict(<class 'list'>, {'0002': ['091700020010'], '0001': ['091700010010']})), ('1063', defaultdict(<class 'list'>, {'0019': ['106300190010'], '0018': ['106300180010']})), ('1126', defaultdict(<class 'list'>, {'0022': ['112600220010'], '0023': ['112600230010']})), ('1242', defaultdict(<class 'list'>, {'0001': ['124200010010'], '0002': ['124200020010']})), ('1686', defaultdict(<class 'list'>, {'0057': ['168600570010'], '0058': ['168600580010']})), ('1695', defaultdict(<class 'list'>, {'0002': ['169500020010'], '0001': ['169500010010']})), ('2302', defaultdict(<class 'list'>, {'0055': ['230200550010'], '0047': ['230200470010'], '0048': ['230200480010'], '0017': ['230200170010'], '0016': ['230200160010'], '000

Similarly, let us order the ScWs by observation times. 

In [169]:
start_times_dt = [Time(mjd, format='mjd').datetime for mjd in scw_start_times]

scw_data = list(zip(scw_ids, scw_versions, scw_start_times, scw_end_times, start_times_dt))

scw_data_sorted = sorted(scw_data, key=lambda x: x[4])

scw_ids_sorted = [x[0] for x in scw_data_sorted]
scw_versions_sorted = [x[1] for x in scw_data_sorted]
scw_start_times_sorted = [x[2] for x in scw_data_sorted]
scw_end_times_sorted = [x[3] for x in scw_data_sorted]

print("Sorted Science Windows by Start Time:")
print(scw_ids_sorted)
print(min(scw_start_times), max(scw_start_times))


Sorted Science Windows by Start Time:
['236700180010', '236800180010', '236700040010', '258100840010', '236700100010', '236800160010', '236600560010', '168600570010', '230200550010', '236700130010', '236700110010', '258100850010', '236900120010', '236600590010', '236800200010', '124200010010', '237400160010', '236900050010', '236700250010', '026601100010', '236900060010', '236600580010', '112600220010', '270100010010', '236600570010', '236900070010', '236900110010', '236600510010', '230200470010', '236600530010', '230200480010', '091700020010', '043200650010', '236600470010', '236600490010', '237300010010', '236700210010', '237300020010', '258200000020', '236600460010', '112600230010', '168600580010', '236700160010', '236700220010', '237400080010', '237200010010', '265000850010', '237200020010', '230200170010', '236800140010', '169500020010', '236900170010', '236800170010', '236900030010', '250600220010', '236600600010', '250600210010', '169500010010', '236600550010', '230200160010', '

In [170]:
scw_pick =  [(s+"."+v).strip() for s,v in zip(scw_ids_sorted,scw_versions_sorted)]

i = 0 # requested index (0 is earliest, -1 is latest)

scw_pick = scw_pick[i]
print(scw_pick)
print(f"Start: {Time(scw_start_times_sorted[i], format='mjd').datetime}, end: {Time(scw_end_times_sorted[i], format='mjd').datetime}, duration:{Time(scw_end_times_sorted[i], format='mjd').datetime-Time(scw_start_times_sorted[i], format='mjd').datetime}")

236700180010.001
Start: 2003-05-29 12:28:09.232338, end: 2003-05-29 13:08:56.233550, duration:0:40:47.001212


Before moving on to using this query for our ODA, we need to input our token. 

In [ ]:
token = ''
token = getpass.getpass('Insert the token')

We also add logging to help visualize the process.

In [ ]:
logging.getLogger().setLevel(logging.INFO) # WARNING, INFO or DEBUG
logging.getLogger('oda_api').addHandler(logging.StreamHandler())

Now let us look at an example.

In [ ]:
disp = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data")

par_dict = {
"E1_keV": "15",
"E2_keV": "20",
"detection_threshold": "5",
"instrument": "isgri",
"osa_version": "OSA10.2",
"product": "isgri_image",
"product_type": "Real",
"scw_list": scw_pick,
'token': token
}
# "integral_data_rights": "all-private" had to be removed as didn't have the rights

data_collection = disp.get_product(**par_dict)
data_collection.show()
data_collection.mosaic_image_0_mosaic.show()

print(data_collection.mosaic_image_0_mosaic.show_meta())


found token in TokenLocation.FILE_CUR_DIR your token payload: {
    "email": "hucklethums@icloud.com",
    "exp": 1741191208,
    "name": "interstellxr",
    "roles": "authenticated user, public-pool-hpc",
    "sub": "hucklethums@icloud.com"
}
found token in TokenLocation.FILE_CUR_DIR your token payload: {
    "email": "hucklethums@icloud.com",
    "exp": 1741191208,
    "name": "interstellxr",
    "roles": "authenticated user, public-pool-hpc",
    "sub": "hucklethums@icloud.com"
}
found token in TokenLocation.FILE_CUR_DIR your token payload: {
    "email": "hucklethums@icloud.com",
    "exp": 1741191208,
    "name": "interstellxr",
    "roles": "authenticated user, public-pool-hpc",
    "sub": "hucklethums@icloud.com"
}
token expires in 217.0 h
token expires in 217.0 h
token expires in 217.0 h
discovered token in environment
discovered token in environment
discovered token in environment
please beware that by default, in a typical setup, oda_api will not output much. To learn how to 

KeyboardInterrupt: 

Finally, let us image our results.

In [ ]:
fits_image = data_collection.mosaic_image_0_mosaic

with fits.open(fits_image) as hdul:
    image_data = hdul[0].data 

# Plot the image
plt.figure(figsize=(10, 8))
plt.imshow(image_data, cmap='inferno', origin='lower', norm=plt.Normalize(vmin=np.percentile(image_data, 5), vmax=np.percentile(image_data, 99)))  
plt.colorbar(label="Counts")
plt.title("INTEGRAL/ISGRI X-ray Image")
plt.xlabel("X Pixel")
plt.ylabel("Y Pixel")
plt.show()